**Happy holidays!**

After putting it on hold for so long, I've finally made my first kaggle submission!



In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

I imported all the training and test data into the  'train_data' and 'test_data' variables and then created a combined variable to concatenate them together. This way I can perform pre-processing and cleaning on both the train and test data and split when it's time to make predictions

In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
combined_df = pd.concat([train_data, test_data], sort=False)

x_feature_columns = ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch','Ticket', 'Fare','Cabin','Embarked', 'Survived']
x_train = combined_df[x_feature_columns]
X_cols = pd.DataFrame(x_train)



I then assigned categorical values 1 and 2 to the two genders

In [ ]:

X_cols['Sex'] =  X_cols['Sex'].replace(['male'], 1)
X_cols['Sex'] =  X_cols['Sex'].replace(['female'], 2)


male = 1
female = 2

This is a simple heatmap to view the correlation between the variables.
Most of the variables do not have any strong correlation to each other and the most notable correlation is between variables'Sex' and 'Survived'. 
This indicates that gender definetely had a role to play in whether one survived the disaster or not

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
cormat =  X_cols.corr()
f,ax = plt.subplots(figsize = (12,10))
sns.heatmap(cormat, cmap = 'coolwarm', annot = True);

Here I created a 'Title' column for the passengers by extracting their titles from their names using the Regex library 

In [ ]:
import regex as re

X_cols['Title'] =  X_cols['Name'].str.extract(r'(?<=\, )(.*?)(?=\.)')


This is a simple table showing how many people with a particular title survived or didn't survived

In [ ]:
pd.crosstab([ X_cols.Title],  X_cols['Survived'], rownames=['Title'])

The titles with the most people are the ones that will have a significant impact on our model. They are: Mr, Mrs, Master and miss. We are simply going to refer to the other titles as 'others'

In [ ]:
X_cols['Title'] = ['Other' if x!='Mr' and x!='Mrs' and x!='Master' and x!='Miss' else x for x in  X_cols.Title]
pd.crosstab([ X_cols.Title],  X_cols['Survived'], rownames=['Title'])



In [ ]:
embarked_mapping = {"S": 0, "C": 1, "Q": 2}
X_cols["Embarked"] = X_cols["Embarked"].map(embarked_mapping)
X_cols.sample(5)

I dropped the Cabin,Ticket and embarked columns as after assessment and trying to find useful patterns, I don't believe they will be of any substantial contribution. I also dropped the 'Name' column since we have extracted from it the more useful 'Title' column

In [ ]:
 X_cols.drop( X_cols.columns[[2,7,9,10]],  axis=1, inplace=True)
 X_cols.sample(5)

Age has null values

In [ ]:
 X_cols.isnull().sum()

I filled the null age values with the mean age in the dataset

In [ ]:
X_cols['Age'].fillna( X_cols['Age'].median(), inplace= True)   
X_cols['Fare'].fillna( X_cols['Fare'].median(), inplace= True) 
X_cols.isnull().sum()

I assigned numerical values to the title column.
Master = 1
Mr = 2
Mrs = 3
Miss = 5
Other = 5

In [ ]:
 X_cols['Title'] =  X_cols['Title'].replace(['Master'], 1)
 X_cols['Title'] =  X_cols['Title'].replace(['Mr'], 2)
 X_cols['Title'] =  X_cols['Title'].replace(['Mrs'], 3)
 X_cols['Title'] =  X_cols['Title'].replace(['Miss'], 4)
 X_cols['Title'] =  X_cols['Title'].replace(['Other'], 5)

In [ ]:
 X_cols['Age'] =  X_cols['Age'].astype(int)
 X_cols['Title'] =  X_cols['Title'].astype(int)
 X_cols.head(10)

age categories 
0 - 20: 1
21 - 40: 2
41 - 60: 3
61- 80: 4
81 - 100: 5

I split the ages into different categories using bins to indicate the range.
I then put these categories into my newly created 'Age Group' Column

In [ ]:
category = pd.cut( X_cols.Age, bins=[0,2,17,65,200], labels =['Baby','Child','Adult','Elderly'])
X_cols.insert(3,'Age Group', category)
X_cols.sample(20)

Age group has 7 null values

In [ ]:
 X_cols.isnull().sum()

In [ ]:
train_data.describe()

I scaled the columns Fare and Age using sci kit learn's MinMaxScaler function

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
  
X_cols[["Fare"]] = scaler.fit_transform( X_cols[["Fare"]])
X_cols[["Age"]] = scaler.fit_transform( X_cols[["Age"]])


X_cols.head(10)

I assigned numerical values again to the categories in age group
baby: 1
child: 2
adult: 3
elderly: 4

In [ ]:
X_cols['Age Group'] =  X_cols['Age Group'].replace(['Baby'], 1)
X_cols['Age Group'] =  X_cols['Age Group'].replace(['Child'], 2)
X_cols['Age Group'] =  X_cols['Age Group'].replace(['Adult'], 3)
X_cols['Age Group'] =  X_cols['Age Group'].replace(['Elderly'], 4)

X_cols.sample(5)

I filled the null age group values with the most frequently occuring catagorical value

In [ ]:
most_common_value = X_cols["Age Group"].value_counts().idxmax()
X_cols['Age Group'] = X_cols['Age Group'].fillna(most_common_value)
X_cols.isnull().sum()

# The combined dataset is now split back into the train and test datasets

In [ ]:
# Split the combined dataset back into the train and test datasets
X_train =  X_cols[ X_cols["Survived"].notnull()]
X_test =  X_cols[ X_cols["Survived"].isnull()]

create a Y column for the dependent variable in the training data 

In [ ]:
Y_train = pd.DataFrame( X_train['Survived'])

I've dropped the 'Survived' column as I need X_train to now contain only independent variables

Drop 'survived' column in X_train as we get ready to create a model for it

In [ ]:
X_train =  X_train.drop(columns=['Survived'])

In [ ]:
 X_train.sample(10)

This boxplot shows the distribution of the ages in the various passenger classes. It can be seen that First class passengers were on average older than the other classes,followed by Second class passengers and then third class having on average the youngest passengers

In [ ]:
sns.boxplot(x = 'Age', y = 'Pclass', data = train_data, orient = 'h')

In [ ]:
Y_train.head(10)

In [ ]:
 Y_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm


model = sm.Logit(Y_train, X_train)
result = model.fit()
print(result.summary())


Using Logistic regression, I got an accuracy of 0.80 on the training dataset

In [ ]:
model=LogisticRegression(max_iter=3000)
Y_train = Y_train.values.ravel()
model.fit(X_train, Y_train)
print('Accuracy=: {:.2f}'.format(model.score(X_train, Y_train)))

In [ ]:
X_test.sample(5)

I then applied the model to my test dataset and got predictions y_pred

In [ ]:
X_test =  X_test.drop(columns=['Survived'])

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_predicted = pd.DataFrame({'Survived': y_pred})

In [ ]:
y_predicted.head()

I read the true y values from the submission dataset to compare to my predicted y values and determine accuracy

In [ ]:
y_true = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [ ]:
y_true =  y_true.drop(columns=['PassengerId'])

In [ ]:
y_true.head()

My model had a score of 0.94 accuracy on the test dataset.

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy: {:.2f}".format(accuracy))

Creating a submission file with just the passengerID and my predicted y values as columns

In [ ]:
submission_df = pd.DataFrame({"PassengerId": X_test["PassengerId"], "Survived": y_predicted['Survived']})

In [ ]:
 submission_df['Survived'] =  submission_df['Survived'].astype(int)

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv("Titanic_submission.csv", index=False)